In [1]:
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow_datasets import ImageFolder

In [2]:
#upload kaggle.json file
from google.colab import files
files.upload()

Saving kaggle.json to kaggle (1).json


{'kaggle.json': b'{"username":"timurzakhoda","key":"ababcf6531560afb23467036bb78d088"}'}

In [3]:
!ls -lga kaggle.json

-rw-r--r-- 1 root 68 Feb 25 04:15 kaggle.json


In [4]:
!pip install -q kaggle

In [5]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

!chmod 600 ~/.kaggle/kaggle.json

In [6]:
!kaggle datasets list

ref                                                       title                                         size  lastUpdated          downloadCount  
--------------------------------------------------------  -------------------------------------------  -----  -------------------  -------------  
michau96/restaurant-business-rankings-2020                Restaurant Business Rankings 2020             16KB  2021-01-30 14:20:45           3261  
gpreda/reddit-wallstreetsbets-posts                       Reddit WallStreetBets Posts                    8MB  2021-02-24 07:59:48           1727  
yorkehead/stock-market-subreddits                         Stock Market Subreddits                        1MB  2021-01-29 13:53:50            337  
aagghh/crypto-telegram-groups                             Crypto telegram groups                       297MB  2021-02-02 09:58:25            199  
timoboz/superbowl-history-1967-2020                       Superbowl History 1967 - 2020                  2KB  2020-02-

In [7]:
!kaggle datasets list -s ASL

ref                                                          title                                               size  lastUpdated          downloadCount  
-----------------------------------------------------------  -------------------------------------------------  -----  -------------------  -------------  
grassknoted/asl-alphabet                                     ASL Alphabet                                         1GB  2018-04-22 19:31:36          18570  
danrasband/asl-alphabet-test                                 ASL Alphabet Test                                   24MB  2018-08-01 04:02:18           1768  
mrgeislinger/asl-rgb-depth-fingerspelling-spelling-it-out    ASL Fingerspelling Images (RGB & Depth)              2GB  2019-12-17 02:52:11            627  
lucasvieirademiranda/aslalphabet                             asl-alphabet                                         2GB  2018-11-23 17:36:46            140  
datamunge/sign-language-mnist                                Sig

In [8]:
!kaggle datasets download -d grassknoted/asl-alphabet    

asl-alphabet.zip: Skipping, found more recently modified local copy (use --force to force download)


In [9]:
!unzip ./asl-alphabet.zip

Выходные данные были обрезаны до нескольких последних строк (5000).
  inflating: asl_alphabet_train/asl_alphabet_train/nothing/nothing19.jpg  
  inflating: asl_alphabet_train/asl_alphabet_train/nothing/nothing190.jpg  
  inflating: asl_alphabet_train/asl_alphabet_train/nothing/nothing1900.jpg  
  inflating: asl_alphabet_train/asl_alphabet_train/nothing/nothing1901.jpg  
  inflating: asl_alphabet_train/asl_alphabet_train/nothing/nothing1902.jpg  
  inflating: asl_alphabet_train/asl_alphabet_train/nothing/nothing1903.jpg  
  inflating: asl_alphabet_train/asl_alphabet_train/nothing/nothing1904.jpg  
  inflating: asl_alphabet_train/asl_alphabet_train/nothing/nothing1905.jpg  
  inflating: asl_alphabet_train/asl_alphabet_train/nothing/nothing1906.jpg  
  inflating: asl_alphabet_train/asl_alphabet_train/nothing/nothing1907.jpg  
  inflating: asl_alphabet_train/asl_alphabet_train/nothing/nothing1908.jpg  
  inflating: asl_alphabet_train/asl_alphabet_train/nothing/nothing1909.jpg  
  inflating

In [32]:
from keras import models
from keras import layers
from keras import optimizers
import keras

In [33]:
from tensorflow.keras.preprocessing import image_dataset_from_directory

In [34]:
@tf.function
def augment(x, y):
    image = tf.image.random_brightness(x, max_delta=0.05)
    image = tf.image.random_flip_left_right(x)
    return image, y

In [35]:
train_dataset= image_dataset_from_directory('./asl_alphabet_train/asl_alphabet_train/',
                                            shuffle=True,
                                            batch_size=128,
                                            image_size=(128,128),
                                            seed = 123,
                                            validation_split=0.1,
                                            subset='training',
                                           )

Found 87000 files belonging to 29 classes.
Using 78300 files for training.


In [36]:
train_dataset = train_dataset.map(augment)

In [37]:
val_dataset= image_dataset_from_directory('./asl_alphabet_train/asl_alphabet_train/',
                                            shuffle=True,
                                            batch_size=128,
                                            image_size=(128,128),
                                            seed = 123,
                                            validation_split=0.1,
                                            subset='validation',
                                           )

Found 87000 files belonging to 29 classes.
Using 8700 files for validation.


In [38]:
from keras.applications import MobileNetV2
base_model = MobileNetV2(weights='imagenet',include_top=False, input_shape = (128,128,3))
base_model.trainable = False
base_model.summary()

Model: "mobilenetv2_1.00_128"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 128, 128, 3) 0                                            
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 64, 64, 32)   864         input_3[0][0]                    
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 64, 64, 32)   128         Conv1[0][0]                      
__________________________________________________________________________________________________
Conv1_relu (ReLU)               (None, 64, 64, 32)   0           bn_Conv1[0][0]                   
_______________________________________________________________________________

In [39]:
model = models.Sequential()
model.add(base_model)
model.add(layers.Flatten())
model.add(layers.Dense(1024, activation='relu'))
model.add(layers.Dense(29, activation='softmax'))

In [40]:
model.compile(optimizer='adam', loss= tf.keras.losses.SparseCategoricalCrossentropy(), metrics=['accuracy'])

In [41]:
%%time
model.fit(train_dataset, epochs=5, verbose=1,validation_data=val_dataset)

Epoch 1/5
612/612 [==============================] - 134s 215ms/step - loss: 3.9850 - accuracy: 0.5467 - val_loss: 0.4612 - val_accuracy: 0.8505
Epoch 2/5
612/612 [==============================] - 127s 206ms/step - loss: 0.4075 - accuracy: 0.8694 - val_loss: 0.3538 - val_accuracy: 0.8816
Epoch 3/5
612/612 [==============================] - 127s 206ms/step - loss: 0.3006 - accuracy: 0.8993 - val_loss: 0.3124 - val_accuracy: 0.8975
Epoch 4/5
612/612 [==============================] - 127s 206ms/step - loss: 0.2320 - accuracy: 0.9227 - val_loss: 0.2833 - val_accuracy: 0.9064
Epoch 5/5
612/612 [==============================] - 126s 204ms/step - loss: 0.2238 - accuracy: 0.9237 - val_loss: 0.2120 - val_accuracy: 0.9259
CPU times: user 7min 56s, sys: 1min 9s, total: 9min 6s
Wall time: 10min 40s


In [42]:
model.evaluate(val_dataset)

68/68 [==============================] - 10s 133ms/step - loss: 0.2120 - accuracy: 0.9259


[0.21195366978645325, 0.9258620738983154]

In [43]:
!pip install -q pyyaml h5py

In [44]:
tf.saved_model.save(model,'/content/drive/MyDrive/')

INFO:tensorflow:Assets written to: /content/drive/MyDrive/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/assets


In [45]:
saved_model_dir = '/content/drive/MyDrive/saved_model.pb'

In [46]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)

In [47]:
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_types = [tf.float16]
model_lite = converter.convert()

INFO:tensorflow:Assets written to: /tmp/tmpi9nwpi5h/assets


INFO:tensorflow:Assets written to: /tmp/tmpi9nwpi5h/assets


In [48]:
with open('/content/drive/MyDrive/model.tflite', 'wb') as f:
    f.write(model_lite)

In [49]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
mobilenetv2_1.00_128 (Functi (None, 4, 4, 1280)        2257984   
_________________________________________________________________
flatten_2 (Flatten)          (None, 20480)             0         
_________________________________________________________________
dense_6 (Dense)              (None, 1024)              20972544  
_________________________________________________________________
dense_7 (Dense)              (None, 29)                29725     
Total params: 23,260,253
Trainable params: 21,002,269
Non-trainable params: 2,257,984
_________________________________________________________________
